# IMDB评分：1代表正面评论，0代表负面评论

In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


#tf.enable_eager_execution()

#!pip install -q tensorflow-datasets

2.8.0


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\pb078553\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete1102VA\imdb_reviews-tr…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\pb078553\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete1102VA\imdb_reviews-te…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\pb078553\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete1102VA\imdb_reviews-un…

Dataset imdb_reviews downloaded and prepared to C:\Users\pb078553\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']


training_sentences = []
training_labels = []

testing_sentences = []  
testing_labels = []


# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
  
  
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
 
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type, padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # 两种展平方式：Flatten()，GlobalAveragePooling1D()
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [7]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs
          , validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.4988 - accuracy: 0.7358 - val_loss: 0.3681 - val_accuracy: 0.8343
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2379 - accuracy: 0.9078 - val_loss: 0.4075 - val_accuracy: 0.8191
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0884 - accuracy: 0.9772 - val_loss: 0.4732 - val_accuracy: 0.8170
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0211 - accuracy: 0.9974 - val_loss: 0.5600 - val_accuracy: 0.8140
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0053 - accuracy: 0.9997 - val_loss: 0.6241 - val_accuracy: 0.8160
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6750 - val_accuracy: 0.8174
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 8.1310e-04 - accuracy: 1.0000 - val_loss: 0.7218 - val_accuracy: 0.8174
Ep

In [8]:
e = model.layers[0]
weights = e.get_weights()[0] # 嵌入层有10000个单词，共16层，因此权重量为10000×16
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [9]:
# 解码器

# 交换字典中的key和value
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all ' ? ? ? ? ? ? ?
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [11]:
import io

out_v = io.open('./tmp/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('./tmp/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num] # 词典中的单词
    embeddings = weights[word_num] # 权重
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
# https://projector.tensorflow.org/

# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs.tsv')
#   files.download('meta.tsv')